# Testing the trained detection model and customizing box naming based on position

## Importing libraries

In [ ]:
import os
import torch, detectron2
import numpy as np
from PIL import Image
import cv2
import pandas as pd

TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)
print("numpy:", np.__version__)
print("pandas:", pd.__version__)

In [ ]:
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

setup_logger()

## Loading metadata and trained model

In [ ]:
train_images = "/home/james/Projects/Fenotypizace/Annotations/Train/images/"
train_json = "/home/james/Projects/Fenotypizace/Annotations/Train/result.json"

register_coco_instances("my_trainset", {}, train_json, train_images)

metadata = MetadataCatalog.get("my_trainset")
dataset_dicts = DatasetCatalog.get("my_trainset")

print(metadata)

In [ ]:
cfg = get_cfg()
# cfg.MODEL.DEVICE = "cpu"
cfg.merge_from_file(
    model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
)
cfg.DATALOADER.NUM_WORKERS = 8
cfg.MODEL.WEIGHTS = "/home/james/Projects/Fenotypizace/output/model_final.pth"
# cfg.MODEL.WEIGHTS = "/home/james/Projects/Fenotypizace/output_old/model_final.pth"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)

## Testing the model on a sample images, to better visualize the locations of the boxes

In [ ]:
test_dir = os.scandir("/home/james/Projects/Fenotypizace/Test_segments/")
save_dir = "/home/james/Projects/Fenotypizace/Segmentations/"

for img in test_dir:
    im = cv2.imread(img.path)
    outputs = predictor(im)
    v = Visualizer(
        im[:, :, ::-1], metadata=metadata, scale=0.8, instance_mode=ColorMode.IMAGE
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imwrite(save_dir + img.name[:-4] + ".png", v.get_image()[:, :, ::-1])

## Creating a functions to customize the box naming based on position

In [ ]:
def num_of_columns(x_min):
    column = "Unknown_Column_"

    if 250 <= x_min <= 460:
        column = "1"

    elif 520 <= x_min <= 740:  # 413
        column = "2"

    elif 760 <= x_min <= 1030:
        column = "3"

    elif 1090 <= x_min <= 1300:
        column = "4"

    elif 1360 <= x_min <= 1625:
        column = "5"

    elif 1640 <= x_min <= 1900:
        column = "6"

    elif 1920 <= x_min <= 2170:
        column = "7"

    elif 2210 <= x_min <= 2470:
        column = "8"

    elif 2510 <= x_min <= 2900:
        column = "9"

    return column

In [ ]:
def num_of_rows(y_min):

    row = "Unknown_Row_"

    if 150 <= y_min <= 350:
        row = "1-"

    elif 380 <= y_min <= 580:
        row = "2-"

    elif 650 <= y_min <= 830:
        row = "3-"

    elif 920 <= y_min <= 1100:
        row = "4-"

    elif 1170 <= y_min <= 1330:
        row = "5-"

    elif 1400 <= y_min <= 1590:
        row = "6-"

    elif 1660 <= y_min <= 1840:
        row = "7-"

    elif 1880 <= y_min <= 2300:
        row = "8-"

    return row

## Creating a function to process the images and save the results

- The function fenotypizace(root, path) takes two arguments: root which is the directory containing the images to be processed, and path which is the directory where the output will be stored.
- The function starts by scanning the root directory and initializing an empty DataFrame with specific columns.
- For each image in the root directory, the function reads the image and applies a prediction model to it.
- If an error occurs during the prediction, the function prints an error message and the name of the file.
- The function then converts the prediction masks to a list that can be processed by the CPU.
- For each mask in the list, the function finds the boundary points of the bounding box, assigns a column and row based on the position of the box, and creates a new directory for storing the box based on its position.
- The function then crops the image and the mask based on the bounding box, combines the cropped image and mask into a new image, and saves the new image in the new directory.
- If the column or row is unknown, the function prints an error message, adds a new row to the DataFrame with the boundary points and the name of the box, and saves the new image with a different name.
- Finally, the function saves the DataFrame to a CSV file in the path directory for debugging purposes.

In [ ]:
def fenotypizace(root, path):
    root_dir = os.scandir(root)
    df = pd.DataFrame(
        columns=["Sloupec_x_min", "Sloupec_x_max", "Radek_y_min", "Radek_y_max", "name"]
    )

    for img in root_dir:
        name = img.name[:-4]
        # print(name)
        # print(img.path)
        # print(img.name[:2])

        try:
            inp = cv2.imread(img.path)
            outputs = predictor(inp)

        except:
            print("Error soubor: " + name)

        masks = np.asarray(
            outputs["instances"].pred_masks.to("cpu")
        )  # Vezme output/obrázek z predikce našeho modelu a udělá z ní list masek, kterou zpracovává CPU
        num_of_iter = len(masks)

        for i in range(num_of_iter):
            item_mask = masks[i]  # Vezme masku pouze jednoho objektu z obrázku
            segmentation = np.where(item_mask == True)

            # Nalezení hraničnch bodů jedlivých boxů
            x_min = int(np.min(segmentation[1]))
            x_max = int(np.max(segmentation[1]))
            y_min = int(np.min(segmentation[0]))
            y_max = int(np.max(segmentation[0]))

            # Funkce na přiřazení sloupce a řádku na základě polohy boxu
            column = num_of_columns(x_min)
            row = num_of_rows(y_min)
            box = row + column
            # print(box)

            # Vytvoření nové složky pro uložení boxu na základě polohy
            new_path = path + "/Tray_" + img.name[:2] + "/" + box + "/"
            # new_path = path + "/Tray_" + img.name[:2] + "/"
            isExist = os.path.exists(new_path)
            if not isExist:
                os.makedirs(new_path)

            # Vyříznutí boxu z obrázku, Vytvoření masky a následné vyřiznutí masky boxu z obrázku
            cropped = Image.fromarray(inp[y_min:y_max, x_min:x_max, :], mode="RGB")
            mask = Image.fromarray((item_mask * 255).astype("uint8"))
            # cv2.imwrite(new_path + name + "_" + str(i) + ".png", np.array(mask))
            cropped_mask = mask.crop((x_min, y_min, x_max, y_max))

            # Spojení boxu a masky do jednoho obrázku, kdy ve předu je obrázek, který je ohraničený maskou = černou barvou
            new_fg_image = Image.new("RGB", cropped_mask.size)
            new_fg_image.paste(cropped, cropped_mask)

            # new_row = pd.DataFrame({"Sloupec_x_min": [x_min], "Sloupec_x_max": [x_max], "Radek_y_min": [y_min], "Radek_y_max": [y_max], "name": [name]})
            # df = pd.concat([df, new_row], ignore_index=True)

            # Uložení vyříznutého boxu
            if column == "Unknown_Column_" or row == "Unknown_Row_":
                print("Chyba pri rozpoznani boxu")
                print(name + " - " + box)
                new_row = pd.DataFrame(
                    {
                        "Sloupec_x_min": [x_min],
                        "Sloupec_x_max": [x_max],
                        "Radek_y_min": [y_min],
                        "Radek_y_max": [y_max],
                        "name": [name + " - " + box],
                    }
                )
                df = pd.concat([df, new_row], ignore_index=True)
                cv2.imwrite(
                    new_path + name + "_" + box + ".png", np.array(new_fg_image)
                )

                # with open(path + "/polohy.txt", "a", encoding="utf-8") as fp:
                #     fp.write("Prvni dvoujcisly je sloupec, druhe je radek\n")
                #     for line in x_min, x_max, y_min, y_max:
                #         if line == y_max:
                #           fp.write("%s"%line + " - " + name +" - " + box + "\n")
                #         else:
                #           fp.write("%s + " %line)

            else:
                cv2.imwrite(
                    new_path + name + "_" + box + ".png", np.array(new_fg_image)
                )  # .png
    df.to_csv(path + "/polohy.csv", index=False)

## Running the function on the sample images

In [ ]:
# root = "/mnt/d/Záloha/Fenotypizace/data/raw/Obrázky/"
# path = "/home/james/Projects/Fenotypizace/Segmentations/"

root = "/home/james/Projects/Fenotypizace/Test_segments/"
path = "/home/james/Projects/Fenotypizace/Masks/"

fenotypizace(root, path)

print("All done")

## Debugging the results

In [ ]:
df = pd.read_csv(path + "/polohy.csv")

In [ ]:
df["name"] = df["name"].astype("category")

df.dtypes

In [ ]:
df

In [ ]:
# filtered_df = df.loc[df['name'] == '48_24-11-21-04-00-01']
# sorted_df = filtered_df.sort_values(by='Radek_y_min')
# sorted_df

In [ ]:
# img = "/home/james/Projects/Fenotypizace/47_29-11-21-10-00-01.png"

# im = cv2.imread(img)
# outputs = predictor(im)
# v = Visualizer(im[:, :, ::-1],
#                 metadata=metadata,
#                 scale=0.8,
#                 instance_mode=ColorMode.IMAGE
# )
# v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
# cv2.imwrite("a.png", v.get_image()[:, :, ::-1])